# Part 1

A description of the problem and a discussion of the background. (15 marks)
Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

This submission will eventually become your Introduction/Business Problem section in your final report. So I recommend that you push the report (having your Introduction/Business Problem section only for now) to your Github repository and submit a link to it.



# Part 2
A description of the data and how it will be used to solve the problem. (15 marks)
Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

This submission will eventually become your Data section in your final report. So I recommend that you push the report (having your Data section) to your Github repository and submit a link to it.

# Part 1 (Problem)

Where in Sprigfield, Illinois would it be best to open a restaurant? Is there a spot in town that has a large number of reviews as compared to others? Is there a certain location that dictates what restaurant is best? Is there a specific type (category of food) of restaurant that is more popular within the city?

This problem will be solved using Foresquare API by looking at different items such as the number of customers and the average amount of money they spent at the restaurant. It will be important to identify trends regarding these metrics with the type of restaurant (category of food) people are most closely associated with.

# Part 2 (Data)

The data that will be used is Foursquare API data for Springfield, Illinois. It will focus on different restaurant type. The popularity of the restaurant (the highest number of patrons and reviews) will be an indicator for how good an area is for a restaurant moving forward. We will display these via a map for the first part of this final assignment. 

In [1]:
!pip install folium

     |████████████████████████████████| 102kB 10.4MB/s ta 0:00:01


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
address = 'Springfield Illinois, Springfield Illinois'

geolocator = Nominatim(user_agent="springfield_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Springfield Illinois is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Springfield Illinois is 39.8023349, -89.651569.


In [4]:
# Set up Foursquare
CLIENT_ID = 'JJX4LXSPN2FL5YMEOCLZBLDUXSZMAXC0TWJBDB2VD45DGKAD' # your Foursquare ID
CLIENT_SECRET = '1DXNJML3EJHPLP1MHQTSMIREHRVVQ4S3AEFK0JTE1VCQKOG5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JJX4LXSPN2FL5YMEOCLZBLDUXSZMAXC0TWJBDB2VD45DGKAD
CLIENT_SECRET:1DXNJML3EJHPLP1MHQTSMIREHRVVQ4S3AEFK0JTE1VCQKOG5


In [5]:
search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [6]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=JJX4LXSPN2FL5YMEOCLZBLDUXSZMAXC0TWJBDB2VD45DGKAD&client_secret=1DXNJML3EJHPLP1MHQTSMIREHRVVQ4S3AEFK0JTE1VCQKOG5&ll=39.8023349,-89.651569&v=20180605&query=Restaurant&radius=50000&limit=100'

In [7]:
results = requests.get(url).json()

In [8]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,False,4e71eadb814dc4209b4dec13,5th Street,US,Springfield,United States,Jefferson,193,"[5th Street (Jefferson), Springfield, IL 62701...","[{'label': 'display', 'lat': 39.80260171474142...",39.802602,-89.649339,62701,IL,The Alamo Restaurant,v-1597420850,NaN
1,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b7597d4f964a520ce162ee3,801 E Monroe St,US,Springfield,United States,at 8th St,643,"[801 E Monroe St (at 8th St), Springfield, IL ...","[{'label': 'display', 'lat': 39.799653, 'lng':...",39.799653,-89.644905,62701,IL,Saputo's Italian Restaurant,v-1597420850,NaN
2,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",1102883,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/maldaners-r...,False,4b75d3d0f964a5209d272ee3,222 S 6th St,US,Springfield,United States,btwn Adams St & Monroe St,398,"[222 S 6th St (btwn Adams St & Monroe St), Spr...","[{'label': 'display', 'lat': 39.80031, 'lng': ...",39.800310,-89.647720,62701,IL,Maldaner's Restaurant,v-1597420850,NaN
3,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f44090a19836ed00191e814,101 W Jefferson St,US,Springfield,United States,NaN,345,"[101 W Jefferson St, Springfield, IL 62702, Un...","[{'label': 'display', 'lat': 39.80343246459961...",39.803432,-89.655350,62702,IL,Mcl Restaurant and Bakery,v-1597420850,NaN
4,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,51c1f6ed498e60fa98fc746b,518 East Adams,US,Springfield,United States,Old State Capitol Plaza,336,"[518 East Adams (Old State Capitol Plaza), Spr...","[{'label': 'display', 'lat': 39.80038045572685...",39.800380,-89.648572,62701,IL,Mangia Italian Restaurant,v-1597420850,NaN


In [15]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Alamo Restaurant,Mexican Restaurant,5th Street,US,Springfield,United States,Jefferson,193,"[5th Street (Jefferson), Springfield, IL 62701...","[{'label': 'display', 'lat': 39.80260171474142...",39.802602,-89.649339,62701,IL,4e71eadb814dc4209b4dec13
1,Saputo's Italian Restaurant,Italian Restaurant,801 E Monroe St,US,Springfield,United States,at 8th St,643,"[801 E Monroe St (at 8th St), Springfield, IL ...","[{'label': 'display', 'lat': 39.799653, 'lng':...",39.799653,-89.644905,62701,IL,4b7597d4f964a520ce162ee3
2,Maldaner's Restaurant,New American Restaurant,222 S 6th St,US,Springfield,United States,btwn Adams St & Monroe St,398,"[222 S 6th St (btwn Adams St & Monroe St), Spr...","[{'label': 'display', 'lat': 39.80031, 'lng': ...",39.800310,-89.647720,62701,IL,4b75d3d0f964a5209d272ee3
3,Mcl Restaurant and Bakery,Bakery,101 W Jefferson St,US,Springfield,United States,NaN,345,"[101 W Jefferson St, Springfield, IL 62702, Un...","[{'label': 'display', 'lat': 39.80343246459961...",39.803432,-89.655350,62702,IL,4f44090a19836ed00191e814
4,Mangia Italian Restaurant,Italian Restaurant,518 East Adams,US,Springfield,United States,Old State Capitol Plaza,336,"[518 East Adams (Old State Capitol Plaza), Spr...","[{'label': 'display', 'lat': 39.80038045572685...",39.800380,-89.648572,62701,IL,51c1f6ed498e60fa98fc746b


In [21]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around downtown

# add a red circle marker to represent the downtown area
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Downtown',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# Explination

Before diving into part two of the project we notice that Italian is the most prevelant type of restaurant in Springfield. However, while many of these are spread out it appears the best place to open a restaurant in is in downtown. 